# Fine-tuning ChatGPT for FHIR Bundle Interpretation
## Notebook Summary:
This notebook serves as a guide to fine-tune the ChatGPT model to specialize in parsing and interpreting FHIR Bundles. The notebook walks you through the initial setup steps, including installing the OpenAI package and importing required libraries. It then guides you through the process of uploading your fine-tuning data and initiating the fine-tuning job. Lastly, it likely contains steps for monitoring and evaluating the fine-tuned model.

## Fine-tuned Effect:
The fine-tuned model is expected to be proficient in understanding and generating text based on FHIR Bundles, which are a standard for healthcare data interchange. This specialized version of ChatGPT could be invaluable in healthcare applications that require natural language interfaces to interact with complex healthcare data structures.


# Install OpenAI Python Package and Import it
In this cell, we install the OpenAI Python package, which provides the interface to interact with the OpenAI API for fine-tuning GPT-3.5.

In [ ]:
!pip install --upgrade openai

## Import Libraries and set the Open AI API key

In [ ]:
import openai 
import os
#Set the API key

openai.api_key = "YOUR_API_KEY"


# Create a Fine-tuning File
In this step, we upload the data file `fhirbundle_training_gpt3.5.jsonl`, which is in the data folder this repository (for reference) for the fine-tuning purpose. This file likely contains FHIR Bundles as inputs and expected Chat GPT outputs.

In [ ]:
testfile = openai.File.create(
  file=open("data/fhirbundle_training_gpt3.5.jsonl", "rb"),
  purpose='fine-tune'
)

In [ ]:
print(testfile)
file_id = testfile['id']
print(file_id)

# Initialize API and Start Fine-tuning
First, we set the OpenAI API key. Then we initiate the fine-tuning job using the file ID obtained earlier.

In [ ]:
#Start a fine-tuning job

fine_tuning_job = openai.FineTuningJob.create( training_file=file_id, model="gpt-3.5-turbo" ) 
print("Fine-tuning job ID:", fine_tuning_job.id)

# Retrive the Fine Tuning Job Status from Open AI

In [ ]:
job_status = openai.FineTuningJob.retrieve(fine_tuning_job.id) 
print("Job Status:", job_status.status)    

# Check Fine-tuning Job Status
This cell retrieves the status of the fine-tuning job using its ID. The status will help us understand whether the fine-tuning has been successful.

In [ ]:
if job_status.status == "succeeded": 
    fine_tuned_model = job_status.fine_tuned_model 
    #Run the Completion API
    #response = openai.Completion.create( model="ft:gpt-3.5-turbo-0613:fly-health::7qmxnF69", prompt="[resourceType] Bundle [entry][0][resource][resourceType] Patient [entry][0][resource][id] 123 [entry][0][resource][name][0][family] Smith [entry][0][resource][name][0][given][0] John [entry][0][resource][gender] male [entry][0][resource][birthDate] 1980-01-01 [entry][1][resource][resourceType] Condition [entry][1][resource][subject][reference] Patient/123 [entry][1][resource][code][coding][0][system] http://snomed.info/sct [entry][1][resource][code][coding][0][code] 44054006 [entry][1][resource][code][coding][0][display] Diabetes mellitus type 2" ) 
    #print("Response:", response.choices[0].text.strip())
    modelid = fine_tuned_model.split(":")[1]
    #Run the ChatCompletion API
    
    completion = openai.ChatCompletion.create(
        model=modelid,
        messages=[
                    {"role": "system", "content": "Given a FHIR Bundle Resource consisting of entries for all the clinical FHIR resources for a single patient, provide a comprehensive medical analysis of the patient."},
                    {"role": "user", "content": "[resourceType] Bundle [entry][0][resource][resourceType] Patient [entry][0][resource][id] 123 [entry][0][resource][name][0][family] Bryan [entry][0][resource][name][0][given][0] John [entry][0][resource][gender] male [entry][0][resource][birthDate] 1980-07-01 [entry][1][resource][resourceType] Condition [entry][1][resource][subject][reference] Patient/123 [entry][1][resource][code][coding][0][system] http://snomed.info/sct [entry][1][resource][code][coding][0][code] 44054006 [entry][1][resource][code][coding][0][display] Diabetes mellitus type 2"}
        ],
        temperature=0.1
    )

    print(completion.choices[0].message)

Once the fine-tuning job has succeeded, this cell tests the fine-tuned model with a sample FHIR Bundle and prints out the response. It uses both the Completion API and the ChatCompletion API to demonstrate different modes of interaction.

In [ ]:
# List 10 fine-tuning jobs
openai.FineTuningJob.list(limit=10)   

# Fine Tine Job Utilitiy Functions
Retrieve Job, List Jobs, Cancel Job, Delete Fine Tuned Model

In [ ]:
# Retrieve the state of a fine-tune
openai.FineTuningJob.retrieve("<YOUR JOB ID HERE>")

In [ ]:
# List up to 10 events from a fine-tuning job
openai.FineTuningJob.list_events(id="<YOUR MODEL ID HERE>", limit=10)

In [ ]:
# Cancel a job
openai.FineTuningJob.cancel("<YOUR JOB ID HERE>")

In [ ]:
# Delete a fine-tuned model (must be an owner of the org the model was created in)
import openai
openai.Model.delete("<YOUR MODEL ID HERE>")